In [15]:
import torch
import torch.nn as nn
import 
u = torch.ones(20)
a_k  = torch.rand(3, 20)
q = torch.rand(4, 3, 20)
T = torch.rand(10,3,20)

In [65]:
a = torch.rand(3,20)
b = torch.rand(3,20)

In [69]:
a

tensor([[0.3124, 0.1562, 0.5107, 0.8983, 0.0520, 0.7503, 0.5575, 0.6284, 0.0793,
         0.9723, 0.8839, 0.0625, 0.0163, 0.1350, 0.0517, 0.7198, 0.2833, 0.8712,
         0.4886, 0.1283],
        [0.2357, 0.4346, 0.6345, 0.8440, 0.2384, 0.2974, 0.5055, 0.3357, 0.5334,
         0.4898, 0.6077, 0.2050, 0.8579, 0.6610, 0.9336, 0.4482, 0.1368, 0.2885,
         0.2795, 0.6843],
        [0.0162, 0.8460, 0.2019, 0.2696, 0.3509, 0.3706, 0.5173, 0.4579, 0.3483,
         0.4532, 0.2130, 0.8266, 0.4980, 0.1375, 0.8521, 0.5419, 0.9995, 0.1191,
         0.3321, 0.7487]])

In [70]:
b

tensor([[0.4747, 0.8068, 0.1269, 0.0041, 0.8436, 0.7613, 0.2404, 0.1955, 0.2217,
         0.7112, 0.7160, 0.7124, 0.7898, 0.4332, 0.5313, 0.8793, 0.9858, 0.3063,
         0.2208, 0.3213],
        [0.1009, 0.3222, 0.3646, 0.7552, 0.1746, 0.3904, 0.5170, 0.8248, 0.2023,
         0.6988, 0.8936, 0.9775, 0.9586, 0.2687, 0.9081, 0.9683, 0.6263, 0.3742,
         0.1440, 0.1255],
        [0.1550, 0.9922, 0.2778, 0.8468, 0.5174, 0.1272, 0.9058, 0.9366, 0.8774,
         0.6354, 0.2829, 0.5397, 0.1173, 0.9174, 0.8432, 0.8830, 0.7356, 0.0077,
         0.2959, 0.1964]])

In [76]:
torch.sum(torch.mul(a,b), dim=1)

tensor([4.0282, 5.5236, 5.7154])

In [77]:
import torch.nn.functional as F
class linear_attention(nn.Module):
    def __init__(self, question_dim, vector_dim, attention_dim):
        super(linear_attention, self).__init__()
        self.fc1 = nn.Linear(question_dim, attention_dim)
        self.fc2 = nn.Linear(vector_dim, attention_dim)
        self.fc3 = nn.Linear(attention_dim, 1)
    def forward(self, question, vector): 
        H = torch.tanh(self.fc1(question)+self.fc2(vector))
        attention_score = F.softmax(self.fc3(H), dim=0)
        x = torch.matmul(attention_score.permute(1,2,0), question.permute(1,0,2))
        return x.squeeze(1)

In [84]:
a = torch.zeros(10,2,requires_grad = True)

In [88]:
a.size(1)

2

In [90]:
a = [torch.tensor([0.2552, 0.1840, 0.1202]), torch.tensor([0.2958, 0.5976, 0.2728]), torch.tensor([0.2336, 0.2327, 0.0548]), torch.tensor([ 0.4371, -0.0618,  0.4259])]

In [92]:
torch.cat(a, 0).view(-1, 4)

tensor([[ 0.2552,  0.1840,  0.1202,  0.2958],
        [ 0.5976,  0.2728,  0.2336,  0.2327],
        [ 0.0548,  0.4371, -0.0618,  0.4259]])

In [78]:
class alternating_co_attention(nn.Module):
    def __init__(self, batch_size, question_dim, vector_dim, attention_dim):
        '''
        input 
        question(time_step, batch_size, dim)
        context(time_step, batch_size, dim)
        '''
        super(alternating_co_attention, self).__init__()
        self.g = torch.zeros(vector_dim)
        self.question_g_attention = linear_attention(question_dim, vector_dim, attention_dim)
        self.context_q_attention = linear_attention(question_dim, vector_dim, attention_dim)
        self.question_c_attention = linear_attention(question_dim, vector_dim, attention_dim)
    def forward(self, question, context):
        temp_vector = self.question_g_attention(question, self.g)
        c_vector = self.context_q_attention(context, temp_vector)
        q_vector = self.question_c_attention(question, c_vector)
        print(q_vector.size())
        print(c_vector.size())
        return q_vector, c_vector

In [64]:
a = alternating_co_attention(3,20,20,20)
a(q,T)

torch.Size([3, 20])
torch.Size([3, 20])


(tensor([[0.4827, 0.6922, 0.4167, 0.6225, 0.6362, 0.6429, 0.7822, 0.5888, 0.3377,
          0.5341, 0.4915, 0.3339, 0.5649, 0.3068, 0.5527, 0.3088, 0.6420, 0.2626,
          0.4300, 0.4787],
         [0.4182, 0.3589, 0.3640, 0.7535, 0.2928, 0.4933, 0.3928, 0.4734, 0.3181,
          0.3022, 0.3632, 0.4339, 0.3391, 0.4615, 0.5991, 0.4541, 0.2787, 0.5443,
          0.8725, 0.5172],
         [0.4249, 0.5454, 0.3598, 0.3986, 0.1710, 0.7588, 0.6043, 0.3123, 0.3966,
          0.5256, 0.7211, 0.4187, 0.3182, 0.3332, 0.4362, 0.6096, 0.6509, 0.5582,
          0.2765, 0.5428]], grad_fn=<SqueezeBackward1>),
 tensor([[0.5088, 0.5281, 0.4568, 0.4724, 0.5017, 0.5946, 0.6394, 0.4605, 0.5893,
          0.4954, 0.5282, 0.4316, 0.5278, 0.4574, 0.5113, 0.4480, 0.6675, 0.4005,
          0.5157, 0.5951],
         [0.1917, 0.4657, 0.3128, 0.4366, 0.4821, 0.4829, 0.3691, 0.5299, 0.6122,
          0.5499, 0.5041, 0.3448, 0.6339, 0.4796, 0.4181, 0.4848, 0.4758, 0.4770,
          0.3082, 0.4788],
         [0.384

In [97]:
import json

In [67]:
import json
from_file = open('validation_features_resnet50.json', 'r', encoding='utf8').read()
images = json.loads(from_file) 

In [69]:
images['empty'] = [[0 for i in range(1000)]]

In [70]:
with open('validation_features_resnet50.json', 'w', encoding='utf8') as f:
        json.dump(images, f, indent=4, ensure_ascii=False) 

In [71]:
images['empty']

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [41]:
import json
from_file = open('train_cleaned.json', 'r', encoding='utf8').read()
recipe = json.loads(from_file) 

In [98]:
def load_cleaned_data(file = 'train_cleaned.json'):
    file = open(file, 'r', encoding='utf8').read()
    recipe = json.loads(file) #json file contains data in str, convert str to dict
    recipe_context = recipe['context']
    recipe_answer = recipe['answer']
    recipe_choice = recipe['choice']
    recipe_question = recipe['question']
    recipe_images = recipe['images']
    return recipe_context, recipe_images, recipe_question, recipe_choice, recipe_answer 

In [99]:
recipe_context, recipe_images, recipe_question, recipe_choice, recipe_answer = load_cleaned_data('train_cleaned.json')

In [100]:
tmp = recipe_context[:3]

[['3 until 5 whole vanilla beans250 gram of vegetable glycerin food gradeevery 100 gram of vanilla beans have 35 until 40 of whole vanilla beans',
  'scrape vanilla beans and get the seeds into vegetable glycerin',
  'vanilla beans seed and vegetable glycerin',
  'whole vanilla beans put in a bottle with seeds and vegetable glycerin'],
 ['there are many variants of borscht this is possibly one of the quickest and easy versions for it you will need\ningredients\n4 cups of chilled sliced beets two 16oz cans\n1 12 cups of beet juice conveniently that is about the amount of juice that will be in the \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 canned beets\n2 tbsp lemon juice\n2 tbsp of sugar\nsalt and pepper to taste\nsour cream\ntools\nblender\xa0\ntbsp\nglass measuring cup to prevent staining\nnice serving bowls\xa0\ncan \xa0opener',
  'open and drain off the beets saveing the juice',
  'place the drained beet in the blender along with\xa0lemon juice and sugar',
  'add the bee

In [43]:
train_images = []
batch_size = 5 
for i in range(0, len(recipe_context), batch_size):
        train_images.append(recipe_images[i : i + batch_size]) 

In [44]:
def transport_1_0_2_image(a):
        max_step = 0
        for i in a:
            if max_step < len(i):
                max_step = len(i)
        new = []
        for i in range(max_step):
            step = []
            for j in a:
                if len(j) <= i:
                    step.append(['empty'])
                else:
                    step.append(j[i])      
            new.append(step)
        return new

In [45]:
x = train_images[0]
x = transport_1_0_2_image(x)

In [46]:
x

[[['how-to-make-halal-vanilla-extract_1_0.jpg'],
  ['how-to-make-cold-borscht_1_0.jpg'],
  [],
  ['purple-gnocchi-with-gorgonzola_1_0.jpg',
   'purple-gnocchi-with-gorgonzola_1_1.jpg',
   'purple-gnocchi-with-gorgonzola_1_2.jpg'],
  ['cabbage-burger_1_0.jpg']],
 [['how-to-make-halal-vanilla-extract_2_0.jpg'],
  ['how-to-make-cold-borscht_2_0.jpg',
   'how-to-make-cold-borscht_2_1.jpg',
   'how-to-make-cold-borscht_2_2.jpg'],
  ['easy-handmade-pumpkin-puree_2_0.jpg',
   'easy-handmade-pumpkin-puree_2_1.jpg'],
  ['purple-gnocchi-with-gorgonzola_2_0.jpg'],
  ['cabbage-burger_2_0.jpg']],
 [['how-to-make-halal-vanilla-extract_3_0.jpg'],
  ['how-to-make-cold-borscht_3_0.jpg', 'how-to-make-cold-borscht_3_1.jpg'],
  ['easy-handmade-pumpkin-puree_3_0.jpg',
   'easy-handmade-pumpkin-puree_3_1.jpg',
   'easy-handmade-pumpkin-puree_3_2.jpg'],
  ['purple-gnocchi-with-gorgonzola_3_0.jpg',
   'purple-gnocchi-with-gorgonzola_3_1.jpg',
   'purple-gnocchi-with-gorgonzola_3_2.jpg',
   'purple-gnocchi-wit

In [52]:
def extract_image_features(batch_images, feature)
    batch = []
    for i in batch_images:
        temp2 = []
        for j in i:
            temp = []
            if len(j) == 0:
                temp = np.zeros((1,1,1000)).tolist()
            for z in j:
                temp.append(feature[z])
            temp2.append(np.sum(temp, axis = 0))
        batch.append(temp2)
    return batch
    

In [58]:
import torch
b = torch.FloatTensor(batch).squeeze(2) 

In [59]:
b.size()

torch.Size([9, 5, 1000])